# **Imports**

In [ ]:
!pip install datasets==2.2.2 #==1.4.1 #==1.18.3
!pip install transformers==4.18.0
#!pip install huggingface_hub==0.1
!pip install torchaudio
!pip install librosa
!pip install jiwer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

# **Wav2Vec2CTCTokenizer**

In [ ]:
from datasets import load_dataset, load_metric, Audio

# (Sorbian Upper-(hsb)-76 MB)
# (Tamil-(ta)-76 MB)
common_voice_train = load_dataset("common_voice", "ta", split="train+validation")
common_voice_test = load_dataset("common_voice", "ta", split="test")

Generating train split:   0%|          | 0/2009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1781 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1779 [00:00<?, ? examples/s]

Generating other split:   0%|          | 0/7428 [00:00<?, ? examples/s]

Generating validated split:   0%|          | 0/12652 [00:00<?, ? examples/s]

Generating invalidated split:   0%|          | 0/594 [00:00<?, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/ta/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e. Subsequent calls will reuse this data.


In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,ஓருயிரும் இரண்டுடலும் நாங்கள்!எம்மை
1,"""எனைஇழந்தேன், உன்னெழிலில் கலந்த தாலே!"""""""
2,நாளையே வெட்டி நடுக்கடலில் போடட்டும்
3,ஒழுக்கத்தின் எய்துவர் மேன்மை இழுக்கத்தின் எய்துவர் எய்தாப் பழி.
4,நினைப்பாலும் வாக்காலும் தேகத் தாலும்
5,எனக்குத் தொண்டை வலிக்கிறது.
6,"""""""தூயநற் காதலர்க்கே - பெருந்"""
7,"""ஞானமுறும் உதாரனிடம் """"அமுத வல்லி"""
8,நைவினை நணுகேல்.
9,மகிழ்ந்து கூத்தாடல் என்றோ?


In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

  0%|          | 0/3788 [00:00<?, ?ex/s]

  0%|          | 0/1781 [00:00<?, ?ex/s]

In [ ]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,இன்பமெனும் பால்நுரையே குளிர் விளக்கே
1,ஊதையினில் துரும்புபோல் அலக்கழிப்போம் பின்னர்
2,மாமலைதான் சென்னி வளைந்து கொடுத்ததுவோ
3,இதன் மொழிபெயர்ப் பென்ன என்றால்
4,தமிழ்க்கவியென் றெனைஅவளும் காத லித்தாள்
5,கந்தன் குடையும் காலிற் செருப்புமாய்
6,தொகையாக எதிர்நிறுத்தித் தூள் தூளாக்கும்
7,நாலைந்து கடவுளின் நற்பெயர் கூறிக்
8,காப்பது விரதம்
9,பூரிக்கும் காதலிலே


In [ ]:
def replace_hatted_characters(batch):
    batch["sentence"] = re.sub('[â]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[î]', 'i', batch["sentence"])
    batch["sentence"] = re.sub('[ô]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[û]', 'u', batch["sentence"])
    return batch

In [ ]:
common_voice_train = common_voice_train.map(replace_hatted_characters)
common_voice_test = common_voice_test.map(replace_hatted_characters)

  0%|          | 0/3788 [00:00<?, ?ex/s]

  0%|          | 0/1781 [00:00<?, ?ex/s]

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'ஃ': 1,
 'அ': 2,
 'ஆ': 3,
 'இ': 4,
 'ஈ': 5,
 'உ': 6,
 'ஊ': 7,
 'எ': 8,
 'ஏ': 9,
 'ஐ': 10,
 'ஒ': 11,
 'ஓ': 12,
 'ஔ': 13,
 'க': 14,
 'ங': 15,
 'ச': 16,
 'ஜ': 17,
 'ஞ': 18,
 'ட': 19,
 'ண': 20,
 'த': 21,
 'ந': 22,
 'ன': 23,
 'ப': 24,
 'ம': 25,
 'ய': 26,
 'ர': 27,
 'ற': 28,
 'ல': 29,
 'ள': 30,
 'ழ': 31,
 'வ': 32,
 'ஷ': 33,
 'ஸ': 34,
 'ஹ': 35,
 'ா': 36,
 'ி': 37,
 'ீ': 38,
 'ு': 39,
 'ூ': 40,
 'ெ': 41,
 'ே': 42,
 'ை': 43,
 'ொ': 44,
 'ோ': 45,
 'ௌ': 46,
 '்': 47,
 '’': 48}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

51

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)


In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
repo_name = "wav2vec2-large-xls-r-300m-ta-colab"

In [ ]:
tokenizer.push_to_hub(repo_name)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:43: FutureWarning: Pass token='wav2vec2-large-xls-r-300m-ta-colab' as keyword args. From version 0.8 passing these as positional arguments will result in an error
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:676: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/AAkhilesh/wav2vec2-large-xls-r-300m-ta-colab into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.6k/1.18G [00:00<?, ?B/s]

Download file runs/Jun02_14-14-09_dbdda1edad4f/1654179356.7264225/events.out.tfevents.1654179356.dbdda1edad4f.…

Download file runs/Jun02_14-14-09_dbdda1edad4f/1654179652.0652208/events.out.tfevents.1654179652.dbdda1edad4f.…

Download file runs/Jun02_14-14-09_dbdda1edad4f/events.out.tfevents.1654179356.dbdda1edad4f.2310.0: 100%|######…

...

Clean file runs/Jun02_14-14-09_dbdda1edad4f/events.out.tfevents.1654179356.dbdda1edad4f.2310.0:   9%|8        …

Clean file runs/Jun03_01-11-47_fe38932e6022/events.out.tfevents.1654218804.fe38932e6022.70.0:  13%|#3        |…

Clean file training_args.bin:  34%|###3      | 1.00k/2.98k [00:00<?, ?B/s]

Clean file runs/Jun03_01-11-47_fe38932e6022/1654218804.738189/events.out.tfevents.1654218804.fe38932e6022.70.1…

Clean file pytorch_model.bin:   0%|          | 1.00k/1.18G [00:00<?, ?B/s]

# **Wav2Vec2FeatureExtractor**

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# **Preprocess Data**

In [ ]:
common_voice_train[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/38a26df4350f0cbefb5a50d4018959b95a7ee56e372bfbcd98491bc12d6599c7/cv-corpus-6.1-2020-12-11/ta/clips/common_voice_ta_19093432.mp3'

In [ ]:
common_voice_train[0]["audio"]

{'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00031871,
        -0.00020725, -0.00080252], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/38a26df4350f0cbefb5a50d4018959b95a7ee56e372bfbcd98491bc12d6599c7/cv-corpus-6.1-2020-12-11/ta/clips/common_voice_ta_19093432.mp3',
 'sampling_rate': 48000}

In [ ]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
common_voice_train[0]["audio"]

{'array': array([ 0.        ,  0.        ,  0.        , ...,  0.0005847 ,
         0.00018592, -0.00039355], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/38a26df4350f0cbefb5a50d4018959b95a7ee56e372bfbcd98491bc12d6599c7/cv-corpus-6.1-2020-12-11/ta/clips/common_voice_ta_19093432.mp3',
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

விலி உனதொரு குடி அடியோடே


In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: சங்காரம் நிசமெனச் சங்கே முழங்கு
Input array shape: (66432,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

  0%|          | 0/3788 [00:00<?, ?ex/s]

  0%|          | 0/1781 [00:00<?, ?ex/s]

In [ ]:
#max_input_length_in_sec = 5.0
#common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

# **Set-up Trainer**

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
cer_metric = load_metric("cer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['project_q.bias', 'quantizer.codevectors', 'project_hid.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'quantizer.weight_proj.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1677: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=200,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

/content/wav2vec2-large-xls-r-300m-ta-colab is already a clone of https://huggingface.co/AAkhilesh/wav2vec2-large-xls-r-300m-ta-colab. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp half precision backend


# **Training**

**put this under inspection console to avoid sleeping**

function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);

In [ ]:
mod = trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3788
  Num Epochs = 200
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 47400


Step,Training Loss,Validation Loss,Wer,Cer
500,4.669300,1.032502,0.940565,0.264796
1000,0.656600,0.695700,0.807503,0.202707
1500,0.416500,0.674416,0.768161,0.183941
2000,0.337500,0.683852,0.771954,0.183050
2500,0.266000,0.722521,0.756218,0.177009
3000,0.232500,0.724277,0.752705,0.179155
3500,0.198800,0.794829,0.746803,0.172603
4000,0.187200,0.755735,0.756077,0.176333
4500,0.174400,0.863372,0.766896,0.182423
5000,0.163600,0.854099,0.728397,0.170639


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1781
  Batch size = 8
Saving model checkpoint to wav2vec2-large-xls-r-300m-ta-colab/checkpoint-500
Configuration saved in wav2vec2-large-xls-r-300m-ta-colab/checkpoint-500/config.json
Model weights saved in wav2vec2-large-xls-r-300m-ta-colab/checkpoint-500/pytorch_model.bin
Feature extractor saved in wav2vec2-large-xls-r-300m-ta-colab/checkpoint-500/preprocessor_config.json
Feature extractor saved in wav2vec2-large-xls-r-300m-ta-colab/preprocessor_config.json
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message

Step,Training Loss,Validation Loss,Wer,Cer
500,4.669300,1.032502,0.940565,0.264796
1000,0.656600,0.695700,0.807503,0.202707
1500,0.416500,0.674416,0.768161,0.183941
2000,0.337500,0.683852,0.771954,0.183050
2500,0.266000,0.722521,0.756218,0.177009
3000,0.232500,0.724277,0.752705,0.179155
3500,0.198800,0.794829,0.746803,0.172603
4000,0.187200,0.755735,0.756077,0.176333
4500,0.174400,0.863372,0.766896,0.182423
5000,0.163600,0.854099,0.728397,0.170639


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1781
  Batch size = 8
Saving model checkpoint to wav2vec2-large-xls-r-300m-ta-colab/checkpoint-6000
Configuration saved in wav2vec2-large-xls-r-300m-ta-colab/checkpoint-6000/config.json
Model weights saved in wav2vec2-large-xls-r-300m-ta-colab/checkpoint-6000/pytorch_model.bin
Feature extractor saved in wav2vec2-large-xls-r-300m-ta-colab/checkpoint-6000/preprocessor_config.json
Feature extractor saved in wav2vec2-large-xls-r-300m-ta-colab/preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xls-r-300m-ta-colab/checkpoint-5000] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored:

In [ ]:
trainer.push_to_hub()

In [ ]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("AAkhilesh/wav2vec2-large-xls-r-300m-ta-colab")
processor = Wav2Vec2Processor.from_pretrained("AAkhilesh/wav2vec2-large-xls-r-300m-ta-colab")

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(repo_name).to("cuda")
processor = Wav2Vec2Processor.from_pretrained(repo_name)

In [ ]:
input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

In [ ]:
common_voice_test_transcription = load_dataset("common_voice", "hsb", data_dir="./cv-corpus-6.1-2020-12-11", split="test")

In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription[0]["sentence"].lower())